In [1]:
import numpy as np # linear algebra
import pandas as pd

df=pd.read_csv('./train.csv')
if(df.isna().any().any()):
    print("Yes")
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
y=df.iloc[:,0].values
X=df.drop(['label'],axis=1)
X=X/255.0

In [3]:
X_copy=np.array(X).reshape(-1,28,28,1)
X_copy.shape

(42000, 28, 28, 1)

In [4]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

y = to_categorical(y, num_classes = 10)

E:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
model=Sequential()
model.add(Conv2D(filters=64,kernel_size=(5,5),padding='Same',activation='relu',input_shape = (28,28,1)))
model.add(Conv2D(filters=64,kernel_size=(5,5),padding='Same',activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 32, kernel_size = (2,2),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 32, kernel_size = (2,2),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [6]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [7]:
model.fit(X_copy, y, 
          batch_size=128, epochs=10, verbose=1)

Epoch 1/10
42000/42000 [==============================] - 599s 14ms/step - loss: 0.4489 - acc: 0.8527
Epoch 2/10
42000/42000 [==============================] - 627s 15ms/step - loss: 0.1164 - acc: 0.9646
Epoch 3/10
42000/42000 [==============================] - 682s 16ms/step - loss: 0.0814 - acc: 0.9763
Epoch 4/10
42000/42000 [==============================] - 650s 15ms/step - loss: 0.0658 - acc: 0.9802
Epoch 5/10
42000/42000 [==============================] - 594s 14ms/step - loss: 0.0540 - acc: 0.9840
Epoch 6/10
42000/42000 [==============================] - 620s 15ms/step - loss: 0.0480 - acc: 0.9862
Epoch 7/10
42000/42000 [==============================] - 597s 14ms/step - loss: 0.0412 - acc: 0.9880
Epoch 8/10
42000/42000 [==============================] - 600s 14ms/step - loss: 0.0395 - acc: 0.9880
Epoch 9/10
42000/42000 [==============================] - 592s 14ms/step - loss: 0.0387 - acc: 0.9886
Epoch 10/10
42000/42000 [==============================] - 593s 14ms/step - loss: 

In [8]:
y_pred=model.predict(X_copy)


In [9]:
y_pred_classes = np.argmax(y_pred,axis = 1)
y_classes=np.argmax(y,axis = 1) 
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_classes,y_pred_classes))

[[4130    0    0    0    0    0    1    0    1    0]
 [   0 4667    3    1    0    0    2    9    2    0]
 [   1    5 4153    2    0    0    1   11    3    1]
 [   0    0    1 4338    0    5    0    2    4    1]
 [   1    4    2    0 4043    0    5    3    1   13]
 [   0    1    0    3    0 3778    6    0    5    2]
 [   2    0    0    0    0    2 4132    0    1    0]
 [   0    3    4    0    0    1    0 4392    0    1]
 [   0    0    2    2    1    0    4    0 4045    9]
 [   3    0    0    1   11    0    1    5    3 4164]]


In [10]:
X_test=pd.read_csv('./test.csv')
X_test=X_test/255.0
X_test=np.array(X_test).reshape(-1,28,28,1)
output=np.argmax(model.predict(X_test),axis=1)


In [11]:
import csv
with open('output2.csv','w',newline='') as f:
    field=['ImageId','Label']
    writer=csv.DictWriter(f,fieldnames=field)
    writer.writeheader()
    for i in range(len(X_test)):
        writer.writerow({'ImageId':i+1,'Label':output[i]})

# Score on Kaggle = 0.99185